In [20]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime, timedelta


In [36]:
def get_aha_journal_days_baack(days_back):
    out = []
    i=0
    while True:
        url =f'https://www.ahajournals.org/topic/featured/featured-article?sortBy=Earliest&startPage={i}&ContentItemType=research-article&pageSize=100'
        i+=1
        driver = webdriver.Chrome()
        driver.get(url)
        html = driver.page_source
        driver.close()
        soup=BeautifulSoup(html,'html.parser')
        divs = soup.find_all('div', class_='col-xs-12')

        for div in divs:   
            try:
                paper_info = {}
                paper_info['abstract'] = div.find('span', class_="hlFld-Abstract").text.replace('            ', '').replace('\n', '')
                paper_info['title'] = div.find('h3', class_='meta__title meta__title__margin').text.replace('                  ', '').replace('\n', '')
                paper_info['paper_id'] = div.find('h3', class_='meta__title meta__title__margin').find('a')['href']
                paper_info['source'] = 'AHA Journal'
                paper_info['date'] = datetime.strptime(div.find('span', class_='meta__pubDate').text,'%d %B %Y' ).strftime('%Y-%m-%d')
                authors = []
                for auth in div.find_all('span', class_='hlFld-ContribAuthor'):
                    authors.append(auth.text)
                response = requests.get(f'https://api.crossref.org/works/{doi}')
                json_resp = response.json()
                affiliations = []
                for affils in json_resp['message']['author']:
                    for affil in affils['affiliation']:
                        affiliations.append(affil['name'])
                affiliations = list(set(affiliations))
                paper_info['authors'] = authors
                paper_info['affiliations'] = affiliations
                paper_info['url']= json_resp['message']['URL']
                paper_info['subjects']=json_resp['message']['subject']
                out.append(paper_info)
            except:
                pass
        if len(out)>0:
            print(out[-1]['date'])
            if datetime.strptime(out[-1]['date'], '%Y-%m-%d').date() <datetime.now().date()-timedelta(days=days_back):
                break
    return pd.DataFrame.from_records(out)

get_aha_journal_days_baack(4)

2023-06-01


,abstract,title,paper_id,source,date,authors,affiliations,url,subjects
0,BACKGROUND:Cognitive dysfunction is a frequent...,Ipsilesional Hippocampal GABA Is Elevated and ...,/doi/10.1161/STROKEAHA.123.043516,AHA Journal,2023-10-01,"[Cristina Torres-López, Maria I. Cuartero, Ali...","[Cardiovascular Division, Department of Medici...",http://dx.doi.org/10.1161/circulationaha.123.0...,"[Physiology (medical), Cardiology and Cardiova..."
1,BACKGROUND:Cardiac conduction is understood to...,Extracellular Perinexal Separation Is a Princi...,/doi/10.1161/CIRCRESAHA.123.322567,AHA Journal,2023-09-29,"[William P. Adams, Tristan B. Raisch, Yajun Zh...","[Cardiovascular Division, Department of Medici...",http://dx.doi.org/10.1161/circulationaha.123.0...,"[Physiology (medical), Cardiology and Cardiova..."
2,BACKGROUND:Heart failure is typical in the eld...,Cardiac Aging Is Promoted by Pseudohypoxia Inc...,/doi/10.1161/CIRCRESAHA.123.322676,AHA Journal,2023-09-29,"[Simone Serio, Christina Pagiatakis, Elettra M...","[Cardiovascular Division, Department of Medici...",http://dx.doi.org/10.1161/circulationaha.123.0...,"[Physiology (medical), Cardiology and Cardiova..."
3,BACKGROUND:Bundled Payments for Care Improveme...,"Changes in Cardiovascular Spending, Care Utili...",/doi/10.1161/CIRCULATIONAHA.123.065109,AHA Journal,2023-10-03,"[Sukruth A. Shashikumar, Jie Zheng, E. John Or...","[Cardiovascular Division, Department of Medici...",http://dx.doi.org/10.1161/circulationaha.123.0...,"[Physiology (medical), Cardiology and Cardiova..."
4,BACKGROUND:The ADAMTS7 locus was genome-wide s...,ADAMTS-7 Modulates Atherosclerotic Plaque Form...,/doi/10.1161/CIRCRESAHA.123.322737,AHA Journal,2023-09-29,"[M. Amin Sharifi, Michael Wierer, Tan An Dang,...","[Cardiovascular Division, Department of Medici...",http://dx.doi.org/10.1161/circulationaha.123.0...,"[Physiology (medical), Cardiology and Cardiova..."
...,...,...,...,...,...,...,...,...,...
95,BACKGROUND:CYP2B6 (cytochrome P450 subfamily I...,Association Between CYP2B6 Polymorphisms and E...,/doi/10.1161/STROKEAHA.122.040507,AHA Journal,2023-07-01,"[Xin Qiu, Yongbo Zhang, Hongqiu Gu, Yingyu Jia...","[Cardiovascular Division, Department of Medici...",http://dx.doi.org/10.1161/circulationaha.123.0...,"[Physiology (medical), Cardiology and Cardiova..."
96,BACKGROUND:Several methods for conducting powe...,Power Analysis for Ordinal Analyses of the Mod...,/doi/10.1161/STROKEAHA.122.041260,AHA Journal,2023-07-01,"[Hannah Johns, Bruce Campbell, Guillaume Turc,...","[Cardiovascular Division, Department of Medici...",http://dx.doi.org/10.1161/circulationaha.123.0...,"[Physiology (medical), Cardiology and Cardiova..."
97,Background:Cardiac allograft vasculopathy (CAV...,Myocardial Flow Assessment After Heart Transpl...,/doi/10.1161/CIRCIMAGING.122.015034,AHA Journal,2023-06-01,"[Kuan-Yin Ko, Chi-Lun Ko, Chii-Ming Lee, Ju-Sh...","[Cardiovascular Division, Department of Medici...",http://dx.doi.org/10.1161/circulationaha.123.0...,"[Physiology (medical), Cardiology and Cardiova..."
98,Background:Existing models for prediction of m...,Machine Learning for Prediction of Adverse Car...,/doi/10.1161/CIRCIMAGING.122.015205,AHA Journal,2023-06-01,"[Ayako Ishikita, Chris McIntosh, Kate Hanneman...","[Cardiovascular Division, Department of Medici...",http://dx.doi.org/10.1161/circulationaha.123.0...,"[Physiology (medical), Cardiology and Cardiova..."


In [40]:
df = pd.read_pickle(r'C:\Users\ethan\Documents\ResearchFinder\oup_temp.pkl')
df

,title,date,paper_id,url,abstract,subjects,authors,affiliations,source
0,The state of play in invasive species policy: ...,2023-10-06,/doi/10.1093/biosci/biad078/7293204,https://academic.oup.com/bioscience/advance-ar...,Despite knowing the threats posed by biologica...,"[iological invasions, invasive species, policy...","[Barney, Jacob N, Haak, David C, Brown, Bryan ...",[Global Change Center's Invasive Species Worki...,Oxford Academic
1,Reconciling paid work and informal caregiving ...,2023-10-06,/doi/10.1093/geroni/igad115/7293241,https://academic.oup.com/innovateage/advance-a...,Background and ObjectivesMany older adults who...,"[Informal Caregiving, employment, older adults]","[Alpass, Fiona, Koreshi, Shanika Yoshini]","[School of Psychology, Massey University , Man...",Oxford Academic
2,Comparison of patients with biopsy positive an...,2023-10-06,/doi/10.1093/rheumatology/kead542/7295624,https://academic.oup.com/rheumatology/advance-...,ObjectiveThere is limited evidence on when to ...,[Primary angiitis of the central nervous syste...,"[Detante, Olivier, Pico, Fernando, Cohort of P...","[Université Caen-Normandie, Internal Medicine,...",Oxford Academic
3,Mapping the individual human cortex using mult...,2023-10-06,/doi/10.1093/braincomms/fcad258/7293252,https://academic.oup.com/braincomms/advance-ar...,Human evolution has seen the development of hi...,"[multidimensional MRI, microstructure imaging,...","[Kundu, Shinjini, Nolan, Amber L, Basser, Pete...",[Department of Laboratory Medicine and Patholo...,Oxford Academic
4,N-Terminal pro-Brain Natriuretic Peptide is a ...,2023-10-06,/doi/10.1093/rheumatology/kead522/7295625,https://academic.oup.com/rheumatology/advance-...,ObjectivesPrediction models based on tradition...,"[Systemic Lupus Erythematosus, Brain Natriuret...","[Huynh, Thao, Grenier, Louis-Pierre, Pineau, C...","[Division of Rheumatology, McGill University ...",Oxford Academic
...,...,...,...,...,...,...,...,...,...
329,Derivation of Expressions for Interdiffusion a...,2023-10-03,/doi/10.1093/oxfmat/itad018/7288186,https://academic.oup.com/ooms/advance-article/...,It has been generally accepted and often menti...,"[Diffusion, Flux, Molar Volume, Drift, Intrins...","[Kulkarni, Kaustubh N]",[Department of Materials Science and Engineeri...,Oxford Academic
330,"Examining the role of diversity, equity, and i...",2023-10-04,/doi/10.1093/ajcp/aqad123/7289074,https://academic.oup.com/ajcp/advance-article/...,ObjectivesThe clinical laboratory workforce pl...,"[burnout, culture, diversity, inclusion, equit...","[Sirintrapun, S Joseph, Garcia, Edna, Soles, R...",[Department of Pathology and Laboratory Medici...,Oxford Academic
331,Grit and Health-Related Quality of Life in Ado...,2023-10-04,/doi/10.1093/jpepsy/jsad066/7289092,https://academic.oup.com/jpepsy/advance-articl...,ObjectiveChallenges with health-related qualit...,"[adherence, AYA, cancer, grit, self-efficacy]","[Zelikovksy, Nataliya, Psihogios, Alexandra M,...","[Department of Medical Social Sciences, Feinbe...",Oxford Academic
332,Space use patterns of two large carnivores: Pu...,2023-10-03,/doi/10.1093/jmammal/gyad085/7288203,https://academic.oup.com/jmammal/advance-artic...,Mammalian carnivores are often vulnerable to t...,"[Atlantic Forest, carnivores, Cerrado, occupan...","[Pônzio, Marcella do Carmo, Pasqualotto, Niels...",[Laboratório de Ecologia e Conservação (LAEC) ...,Oxford Academic
